In [108]:
from datetime import datetime
import glob
import numpy as np
import pandas as pd
from pydantic import BaseModel
from python_graphql_client import GraphqlClient
from textwrap import dedent
from typing import Dict, Optional

import import_ipynb

In [55]:
GRAPHQL_ENDPOINT = "http://localhost:8000/ingest/graphql"

client = GraphqlClient(endpoint=GRAPHQL_ENDPOINT)

In [56]:
class BaseMutation(BaseModel):
    def gen_variables(self):
        return self.dict()

    def execute(self, client):
        query = self.gen_mutation()
        variables = self.gen_variables()
        return client.execute(query=query, variables=variables)


class BaseQuery(BaseModel):
    def gen_variables(self) -> Dict:
        return {}

    def execute(self, client):
        query = self.gen_query()
        variables = self.gen_variables()
        response = client.execute(query=query, variables=variables)
        return response["data"][self.query_name]

In [84]:
class GetTradesByAccountId(BaseQuery):
    accountId: str
        
    @property
    def query_name(self):
        return "tradesByAccountId"
    
    def gen_variables(self) -> Dict:
        return dict(accountId=self.accountId)
    
    def gen_query(self) -> str:
        return dedent("""
        query x ($accountId: String!, $limit: Int) {
            tradesByAccountId(accountId: $accountId, limit: $limit) {
                id,
                accountId,
                fifoPnlRealized,
                transactionId,
                executedAt,
                groupSet { id, name }
            }
        }
        """)

In [85]:
x = GetTradesByAccountId(accountId="U2888710", limit=100000000)
trades = x.execute(client)
for x in trades:
    x["groups"] = [xx["name"] for xx in x["groupSet"]]
    if len(x["groups"]) == 1:
        x["groups"] = x["groups"][0]

In [109]:
df = pd.DataFrame(trades)
df.executedAt = pd.to_datetime(df.executedAt)
df = df[ df["groups"].str.len() > 0 ]

In [110]:
df.tail()

In [ ]:
# @TODO
# 
# At the portfolio level, what was the Portfolio Theta daily/weekly/monthly?
# At the group level, what was the Portfolio Theta daily/weekly/monthly?
# 
# Create a plot of the cashflows
# 
# How much did small trades benefit year end numbers?
# 
# What's the skew and convexity?
# 
# What's the covariance matrix between the top X groups?
# How did that change over the year?
# What's healthy - where are we headed?